# Tutorium 2

## Mathematisches Modell

**Zielfunktion**

\begin{equation}
	minimiere\ \ Z = \sum^{I}_{i=1} f_{i} \cdot y_{i} + \sum^{I}_{i=1} \sum^{J}_{j=1} c_{ij} \cdot x_{ij}
\end{equation}

**unter den Nebenbedingungen**

\begin{align}
&& \sum^{I}_{i=1} x_{ij} &= d_{j} && \forall j \in J \\[5pt]
&& \sum^{J}_{j=1} x_{ij} &\leq b_{i} \cdot y_{i} && \forall i \in I \\[10pt]
&& x_{ij} &\geq 0 && \forall i \in I, \forall j \in J \\[5pt]
&& y_{i} &\in \left\{ 0, 1 \right\} &&
\end{align}


## Aufgabe 2c)
Nutzen Sie zur Formulierung von Zielfunktion und Nebenbedingungen die mengenbasierte Syntax.

In [ ]:
import gurobipy as gp
from gurobipy import GRB

### Parameter:

| Absatzort | Bedarfsmenge |   |   Standort | Kapazität | Fixkosten |
|----------:|:------------:|:-:|-----------:|:---------:|:---------:|
| Frankfurt |      200     |   |    Dresden |    400    |   60000   |
|    Berlin |      250     |   |     Bremen |    350    |   60000   |
|   München |      150     |   | Düsseldorf |    600    |   80000   |
|      Köln |      300     |   |            |           |           |
|   Hamburg |      100     |   |            |           |           |

In [ ]:
d = [200, 250, 150, 300, 100]
b = [400, 350, 600]
f = [60000, 60000, 80000]

Transportkosten zwischen Standort i und Absatzort j in GE/ME:

|  Ort i \ j | Frankfurt | Berlin | München | Köln | Hamburg |
|-----------:|:---------:|:------:|:-------:|:----:|:-------:|
|    Dresden |    270    |   170  |   400   |  450 |   300   |
|     Bremen |    320    |   280  |   490   |  230 |   120   |
| Düsseldorf |    120    |   390  |   240   |  230 |   370   |

In [ ]:
c = [[270, 170, 400, 450, 300],
     [320, 290, 490, 230, 120],
     [120, 390, 240, 230, 370]]

Definition der Mengen:

In [ ]:
I_max = len(b)
J_max = len(d)

I = range(I_max)
J = range(J_max)

Initialisierung des Modells:

In [ ]:
m = gp.Model()

### Initialisierung der Variablen:

Variante - Wiederholter Aufruf von `model.addVar()`

In [ ]:
x = {}
for i in I:
     for j in J:
          x[i,j] = m.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(i)+str(j))

In [ ]:
y = {}
for i in I:
     y[i] = m.addVar(vtype=GRB.BINARY, name="y_"+str(i))

Variante - Aufruf von `model.addVars()` 

In [ ]:
x = m.addVars(I_max, J_max, vtype=GRB.CONTINUOUS, name="x")

In [ ]:
y = m.addVars(I_max, vtype=GRB.BINARY, name="y")

Definition der Zielfunktion

In [ ]:
m.setObjective(gp.quicksum(f[i] * y[i] for i in I) + gp.quicksum(c[i][j] * x[i,j] for j in J for i in I), GRB.MINIMIZE)

### Hinzufügen der Nebenbedingungen

Variante - Wiederholter Aufruf von `model.addConstr()`

In [ ]:
for j in J:
     m.addConstr(gp.quicksum(x[i,j] for i in I) == d[j], "nb_" + str(j))

In [ ]:
for i in I:
     m.addConstr(gp.quicksum(x[i,j] for j in J) <= b[i] * y[i], "kb_" + str(i))

Variante - Aufruf von `model.addConstrs()`

In [ ]:
m.addConstrs((x.sum("*", j) == d[j] for j in J), name="nb")

In [ ]:
m.addConstrs((x.sum(i, "*") == b[i] * y[i] for i in I), name="kb")

Hinweis: `var.sum()` funktioniert nur, wenn vorher `var` mit `model.addVars()` erzeugt wurde, da `model.addVars()` ein Tupledict zurückgibt und `var.sum()` nur für Tupledicts definiert ist. 

### Optimierung:

In [ ]:
m.optimize()

Ergebnisausgabe:

In [ ]:
m.printAttr(GRB.Attr.ObjVal)

In [ ]:
m.printAttr(GRB.Attr.X)

## Aufgabe 2d)
Angenommen das Unternehmen besitzt keine Lieferverpflichtung und erhält pro Transport eine Entschädigung von 350 GE. Welche Transporte würden dann ausgeführt
werden, wo werden Standorte gebaut und wie hoch ist der Gewinn? Ändern sie hierfür die erforderlichen Nebenbedingungen und die Zielfunktion und optimieren Sie erneut.

Keine Lieferverpflichtung:
Nachfrage pro Standort muss nicht mehr vollständig gedeckt sein.

In [ ]:
for j in J:
     m.addConstr(gp.quicksum(x[i,j] for i in I) <= d[j], "nb_" + str(j))

Transportentschädigung: Gewinn wird mit variablen Transportkosten verrechnet.

In [ ]:
g = 350
gewinn = gp.quicksum((g - c[i][j]) * x[i,j] for j in J for i in I)
fixeKosten = gp.quicksum(f[i] * y[i] for i in I)
m.setObjective(gewinn - fixeKosten, GRB.MAXIMIZE)